In [9]:
import pandas as pd
import mysql.connector as mysql
import re
import numpy as np

db = mysql.connect(
    host="localhost",
    user="root",
    password="password",
    port="3306",
    database="serlo"
)


def comment_with_link(content):
    if content == None or content == "NULL":
        return None
    
    return "http://" in content or "https://" in content

def query(sql):
    c = db.cursor()
    c.execute(sql)
    return c.fetchall()


def comment_has_link(user_id):
    user_comments = query('''select comment_vote.user_id, comment.content
    from comment_vote
    inner join comment
    on comment.id = comment_vote.comment_id
    where comment_vote.user_id = comment.author_id''' )

    for user in user_comments:
       if user[0] == user_id and comment_with_link(user[1]):
            return user[1]
    return False
    
    
df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df.head()
df["comment_has_link"] = df["id"].map(comment_has_link)  
df_potential_bot = df[
    ((df["edits"] + df["reviews"] + df["taxonomy"] == 0) & df["comment_has_link"])
].copy()
df_potential_bot.sort_values(["id"], inplace=True)
df_potential_bot



/Users/kellyshor/.asdf/installs/python/3.9.1/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,username,description,date,edits,reviews,comments,taxonomy,comment_has_link
2990,61630,amitsulu,NULL,2016-08-30 06:49:01,0.0,0.0,3.0,0.0,â€¢You may determine out quite a few #Packers ...
13834,179044,robinsmithsters,NULL,2020-09-25 11:46:22,0.0,0.0,3.0,0.0,Higher education is a lofty endeavor with as m...
